# RadCAD Model

In [2]:
import random
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
pd.options.plotting.backend = "plotly"
from dataclasses import dataclass
import copy
from dataclasses import field
from radcad import Model, Simulation, Experiment
from radcad.engine import Engine, Backend

In [3]:
METERS = int
COINS = int
PERCENTAGE = float   


#utils
def default(obj):
    return field(default_factory=lambda: copy.copy(obj))


@dataclass
class Parameters:
    # crash_chance is the chance of crashing in the beginning
    crash_chance: PERCENTAGE = default([10])

    # crash_increase is by how much the difficulty_factor will increase each timestep
    crash_increase: PERCENTAGE = default([1])


# Initialize Parameters instance with default values
system_params = Parameters().__dict__


@dataclass
class StateVariables:
    distance: METERS = 0
    coins: COINS = 0
    difficulty_factor: int =0
    player_crashes: int = 0

initial_state = StateVariables().__dict__



def p_sprint(params, substep, state_history, prev_state, **kwargs):
    '''Calculates the amount of distance covered per timestep'''

    if prev_state['player_crashes']==1:
        distance_covered = 0
    else:
        distance_covered=5
    
    return {'distance_covered': distance_covered}

def p_difficulty(params, substep, state_history, prev_state, **kwargs):
    '''Calculates the increase in difficulty every timestep'''

    if prev_state['timestep']<1:
        difficulty_increase=0

    # if player crashed in previous step then dont increase difficulty factor
    elif prev_state['player_crashes'] == 1:
        difficulty_increase=0
    
    else:
        #Every second timestep increase difficulty by 1
        difficulty_increase = 1 if prev_state['timestep']%2==1 else 0

    return {'difficulty_increase': difficulty_increase}


def p_generate_coins(params, substep, state_history, prev_state, **kwargs):
    '''Calculates the amount of coins generated'''

    if prev_state['distance']<1:
        new_coins=0

    # if player crashed in previous step then dont increase coins
    elif prev_state['player_crashes'] == 1:
        new_coins=0
    
    else:
        # if distance is less than 50 mint 1 coin, if its less than 100 mint 2, if its above 100 mint 3
        distance = prev_state['distance']
        new_coins = 1 if distance < 50 else 2 if distance < 100 else 3

    return {'new_coins': new_coins}


def p_crash(params, substep, state_history, prev_state, **kwargs):
    '''Calculates the probability of crash'''

    if prev_state['difficulty_factor']<2:
        player_crashed=0

    elif prev_state['player_crashes']==1:
        player_crashed=1
    
    else: 
        # Take the initial chance of crashing and adding it with the current difficulty factor to update the chance of crash
        crash_chance = (params['crash_chance'] + prev_state['difficulty_factor'])/100
        # If the random number generated between 0 and 1 is smaller than the percentage chance of crashing then we assume the player crashed
        if random.random()< crash_chance:
            player_crashed = 1 
        else:
            player_crashed=0

    return {'player_crashed': player_crashed}



def s_update_distance(params, substep, state_history, prev_state, policy_input, **kwargs):
    '''Update the state of the distance variable by the amount of distance sprinted'''

    updated_distance = np.ceil(prev_state['distance'] + policy_input['distance_covered'])
    return ('distance', max(updated_distance, 0))


def s_update_coins(params, substep, state_history, prev_state, policy_input, **kwargs):
    '''Update the state of the coins variable by the amount of new coins generated'''

    updated_coins = np.ceil(prev_state['coins'] + policy_input['new_coins'])
    return ('coins', max(updated_coins, 0))


def s_update_difficulty(params, substep, state_history, prev_state, policy_input, **kwargs):
    '''Update the state of the difficulty variable by the amount of difficulty increase'''

    updated_difficulty = np.ceil(prev_state['difficulty_factor'] + policy_input['difficulty_increase'])

    return ('difficulty_factor', max(updated_difficulty, 0))

def s_update_crash(params, substep, state_history, prev_state, policy_input, **kwargs):
    '''Update the state of the crash variable'''

    updated_crash = policy_input['player_crashed']

    return ('player_crashes', max(updated_crash, 0))

###

state_update_blocks = [
    {
        'policies': {
            'p_sprint': p_sprint,
            'p_difficulty': p_difficulty,
            'p_generate_coins':p_generate_coins,
            'p_crash':p_crash,
        },
        'variables': {
            'distance': s_update_distance,
            'coins': s_update_coins,
            'difficulty_factor':s_update_difficulty,
            'player_crashes': s_update_crash

        }
    },

]



In [4]:

# config and run

#number of timesteps
TIMESTEPS = 40
#number of monte carlo runs
RUNS = 5


model = Model(initial_state=initial_state, state_update_blocks=state_update_blocks, params=system_params)
simulation = Simulation(model=model, timesteps=TIMESTEPS, runs=RUNS)

experiment = Experiment(simulation)
# Select the Pathos backend to avoid issues with multiprocessing and Jupyter Notebooks
experiment.engine = Engine(backend=Backend.PATHOS)

result = experiment.run()


df = pd.DataFrame(result)
df

,distance,coins,difficulty_factor,player_crashes,simulation,subset,run,substep,timestep
0,0.0,0.0,0.0,0,0,0,1,0,0
1,5.0,0.0,0.0,0,0,0,1,1,1
2,10.0,1.0,1.0,0,0,0,1,1,2
3,15.0,2.0,1.0,0,0,0,1,1,3
4,20.0,3.0,2.0,0,0,0,1,1,4
...,...,...,...,...,...,...,...,...,...
200,40.0,7.0,4.0,1,0,0,5,1,36
201,40.0,7.0,4.0,1,0,0,5,1,37
202,40.0,7.0,4.0,1,0,0,5,1,38
203,40.0,7.0,4.0,1,0,0,5,1,39


In [5]:
TIMESTEPS = 30
RUNS = 100

model = Model(initial_state=initial_state, state_update_blocks=state_update_blocks, params=system_params)
simulation = Simulation(model=model, timesteps=TIMESTEPS, runs=RUNS)

def median_coin_100_runs(i):
    simulation.model.params.update({
        # Running a parameter sweep on the initial chance of crashing 
        'crash_chance':[i]
    })
    experiment = Experiment(simulation)

    # Select the Pathos backend to avoid issues with multiprocessing and Jupyter Notebooks
    experiment.engine = Engine(backend=Backend.PATHOS)

    result = experiment.run()


    df = pd.DataFrame(result)

    median_coins = df['coins'].median()
    return median_coins

In [6]:
median_coins_expected = 5
crash_chance_initial_guess = 1
step_distance = 3


guess = crash_chance_initial_guess

for i in range(20):
    median_for_guess = median_coin_100_runs(guess)
    print(i,', crash chance = ',guess, ', median_for_guess = ', median_for_guess)
    if median_for_guess>median_coins_expected:
        guess+=step_distance
    else:
        break

print(guess)

0 , crash chance =  1 , median_for_guess =  11.0
1 , crash chance =  4 , median_for_guess =  9.0
2 , crash chance =  7 , median_for_guess =  7.0
3 , crash chance =  10 , median_for_guess =  6.0
4 , crash chance =  13 , median_for_guess =  6.0
5 , crash chance =  16 , median_for_guess =  6.0
6 , crash chance =  19 , median_for_guess =  5.0
19


# Language model connection

In [7]:
import openai
import os 
import json
# from dotenv import load_dotenv

In [12]:
openai.api_key = ''

## Toolkit

In [207]:
# tools in the tool kit

def change_param(param,value):
    '''Changes the value of a parameter in the model'''
    # value = param
    return f'new {param} value is {value} and simulation run'

def model_info(param):
    return f'{param} = 3'

def analyze_dataframe(question):
    return 'analyzed dataframe'

In [14]:
# list of parameters of the cadcad model
listofparams = system_params.keys()

In [193]:
# tool descriptions

function_descriptions_multiple = [
    {
        "name": "change_param",
        "description": "Changes the parameter of the cadcad simulation and returns dataframe as a global object. The parameter must be in this list:" + str(listofparams),
        "parameters": {
            "type": "object",
            "properties": {
                "param": {
                    "type": "string",
                    "description": "parameter to change. choose from the list" + str(listofparams),
                },
                "value": {
                    "type": "string",
                    "description": "value to change the parameter to, eg. 0.1",
                },
            },
            "required": ["param", "value"],
        },
    },
    {
        "name": "model_info",
        "description": "print current state of the simulation parameters",
        "parameters": {
            "type": "object",
            "properties": {
                "param": {
                    "type": "string",
                    "description": "type of information to print. choose from the list: " + str(listofparams),
                },
            },
            "required": ["param"],
        },
    },
    {
        "name": "analyze_dataframe",
        "description": "Use this whenever a quantitative question is asked about the dataframe",
        "parameters": {
            "type": "object",
            "properties": {
                "question": {
                    "type": "string",
                    "description": "The question asked by user that can be answered by an LLM dataframe agent",
                },
            },
            "required": ["question"],
        },
    },
]

In [194]:
def executor_agent(prompt):
    """Give LLM a given prompt and get an answer."""

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": prompt}],
        # add function calling
        functions=function_descriptions_multiple,
        function_call="auto",  # specify the function call
    )

    output = completion.choices[0].message
    return output


user_prompt = "whats the current value of crash chance?"
print(executor_agent(user_prompt))

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "model_info",
    "arguments": "{\n  \"param\": \"crash_chance\"\n}"
  }
}


In [56]:
user_prompt = "what is the avg value of the second row in the simulation?"
print(executor_agent(user_prompt))

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "analyze_dataframe",
    "arguments": "{\n  \"question\": \"avg value of second row\"\n}"
  }
}


In [141]:
user_prompt = "Change the parameter of crash chance to 20"
answer = executor_agent(user_prompt)

In [135]:
def planner_agent(prompt):
    """Give LLM a given prompt and get an answer."""

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {
            "role": "system",
            "content": '''
            You will be provided with a question by the user that is trying to run a cadcad python model. Your job is to provide the set of actions to take to get to the answer using the functions available.
            For example, if the user asks "if my crash chance parameter was 0.2, what would the avg coins be at the end of the simulation?" you reply with "### 1) we use the function change_param to change the crash chance parameter to 0.2,\n 2) use the function analyze_dataframe to get the avg coins at the end of the simulation. ###" 
            if the user asks "what would happen to the coins at the end of the simulation if my crash chance param was 10 perc lower?" you reply with "### 1) find out the current value of crash chance param using the model_info function,\n 2) we use function change_param to change the crash chance parameter to 0.1*crash_chance .\n 3) we use function analyze_dataframe to get the avg coins at the end of the simulation. ###"
            These are the functions available to you: {function_descriptions_multiple}. always remember to start and end plan with ###.
            '''
            },
            {
            "role": "user",
            "content": prompt
            }
        ],
    )

    output = completion.choices[0].message
    return output

In [114]:
answer = planner_agent("whats the current value of crash chance, increase it by 5 perc and tell me the avg coins at the end of the simulation?")
print(answer.content)

### 1) Use the function model_info to find out the current value of crash chance param.
2) Use the function change_param to increase the crash chance parameter by 5 percent.
3) Use the function analyze_dataframe to get the avg coins at the end of the simulation. ###


In [115]:
# plan parser function which takes a string and returns a list of functions to call. It uses the \n as a delimiter to split the string into a list of functions to call.
def plan_parser(plan):
    plan = plan.split('###')[1]
    plans = plan.split('\n')
    return plans

In [188]:
new_plan=['hi','hello']
new_plan[0]

'hi'

In [200]:
# pritn with colors
def print_color(string, color):
    print("\033["+color+"m"+string+"\033[0m")


# Orchestrator

In [205]:
def orchestrator_pipeline(user_input):
    plan = planner_agent(user_input).content
    plan_list = plan_parser(plan)
    print_color("Planner Agent:", "32")
    print('I have made a plan to follow: \n')

    for plan in plan_list:
        print(plan)

    print('\n')
    for plan in plan_list:
        print_color("Executor Agent:", "31")
        print('Thought: My task is to', plan)
        answer = executor_agent(plan)
        print('Action: I should call', answer.function_call.name,'function with these' , json.loads(answer.function_call.arguments),'arguments')
        print('Observation: I got this answer', eval(answer.function_call.name)(**json.loads(answer.function_call.arguments)))


In [206]:
orchestrator_pipeline("whats the current value of crash chance, increase it by 5 perc and tell me the avg coins at the end of the simulation?")

Planner Agent:
I have made a plan to follow: 

 1) To find out the current value of the crash chance parameter, we use the model_info() function.
2) Use the change_param() function to increase the crash chance parameter by 5%.
3) Use the analyze_dataframe() function to get the average coins at the end of the simulation. 


Executor Agent:
Thought: My task is to  1) To find out the current value of the crash chance parameter, we use the model_info() function.
Action: I should call model_info function with these {'param': 'crash_chance'} arguments
Observation: I got this answer crash_chance = 3
Executor Agent:
Thought: My task is to 2) Use the change_param() function to increase the crash chance parameter by 5%.
Action: I should call change_param function with these {'param': 'crash_chance', 'value': '0.05'} arguments
Observation: I got this answer new crash_chance value is 0.05 and simulation run
Executor Agent:
Thought: My task is to 3) Use the analyze_dataframe() function to get the a

In [ ]:
# execute code here 
# what i can do is make a list of known values and add them each time a task is done.

In [52]:
answer2 = answer.content

In [53]:
answer2

'1) Use the function `model_info` to find out the current value of crash chance parameter.\n2) Use the function `change_param` to increase the crash chance parameter by 5 percent.\n3) Use the function `analyze_dataframe` to get the average coins at the end of the simulation.'